In [ ]:
"""Module reads stock price history from yfinance and dumps in DB"""

import datetime
import yfinance as yf
from sqlalchemy import create_engine
from stocknamedata import scriptNameList

PG_USER = "postgres"
PG_PASSWORD = "data123"
PG_HOSTNAME = "127.0.0.1"
PG_PORT = "5432"
PG_DB_NAME = "dailypricedata"

connect = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOSTNAME}:{PG_PORT}/{PG_DB_NAME}"


start = datetime.datetime(2001, 1, 1)
end = datetime.datetime.now()

In [ ]:
def to_alchemy(anydbengine, stockname, dataframe):
    """dumps data frame to database"""
    print(stockname)
    dataframe.to_sql(
        stockname,
        con=anydbengine,
        index=False,
        if_exists='replace'
    )

In [ ]:
# Establishing the connection

engine = create_engine(connect)

for script_name in scriptNameList:
    stock_price_info_Frame = yf.download(
        script_name, start=start, end=end, progress=False).reset_index()
    stock_price_info_Frame = stock_price_info_Frame.rename(columns={
        "Date": "Date",
        "Open": "Open",
        "High": "High",
        "Low": "Low",
        "Close": "Close",
        "Adj Close": "AdjClose",
        "Volume": "Volume"
    })
    script_name = script_name.replace(".NS", "")
    to_alchemy(engine, script_name, stock_price_info_Frame)